In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from livelossplot.inputs.tf_keras import PlotLossesCallback
from keras.utils.vis_utils import plot_model


In [ ]:
for ex in os.listdir('drive/MyDrive/Colab Notebooks/Take 2/train'):
    print(str(len(os.listdir("drive/MyDrive/Colab Notebooks/Take 2/train/" + ex))) + " " + ex + " images")

7215 happy images
4965 neutral images
3171 surprise images
4830 sad images
4097 fear images
4138 angry images
436 disgust images


In [ ]:
img = 48
batch = 64
data_train = ImageDataGenerator(horizontal_flip=True)
train_gen = data_train.flow_from_directory("drive/MyDrive/Colab Notebooks/Take 2/train",
                                                    target_size=(img,img),
                                                    color_mode="grayscale",
                                                    batch_size=batch,
                                                    class_mode='categorical',
                                                    shuffle=True)
data_test = ImageDataGenerator(horizontal_flip=True)
test_gen = data_test.flow_from_directory("drive/MyDrive/Colab Notebooks/Take 2/test",
                                                    target_size=(img,img),
                                                    color_mode="grayscale",
                                                    batch_size=batch,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 28852 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
face_model = Sequential()

# 1 - Convolution
face_model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
face_model.add(BatchNormalization())
face_model.add(Activation('relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))

# 2 - Convolution
face_model.add(Conv2D(128,(5,5), padding='same'))
face_model.add(BatchNormalization())
face_model.add(Activation('relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))

# 3 - Convolution
face_model.add(Conv2D(512,(3,3), padding='same'))
face_model.add(BatchNormalization())
face_model.add(Activation('relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))

# 4 - Convolution
face_model.add(Conv2D(512,(3,3), padding='same'))
face_model.add(BatchNormalization())
face_model.add(Activation('relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))

# Flattening
face_model.add(Flatten())

# 1 - Fully connected layer 
face_model.add(Dense(256))
face_model.add(BatchNormalization())
face_model.add(Activation('relu'))
face_model.add(Dropout(0.25))

# 2 - Fully connected layer
face_model.add(Dense(512))
face_model.add(BatchNormalization())
face_model.add(Activation('relu'))
face_model.add(Dropout(0.25))

face_model.add(Dense(7, activation='softmax'))
opt = Adam(learning_rate=0.0005)
face_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
face_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_24 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 24, 24, 64)       0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 24, 24, 64)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 24, 24, 128)      

In [1]:
%%time
epochs = 15
steps = train_gen.n//train_gen.batch_size
test_steps = test_gen.n//test_gen.batch_size
dec_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')
cp = ModelCheckpoint("model_weights.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
callbacks = [PlotLossesCallback(), cp, dec_lr]
history = face_model.fit(
    x=train_gen,
    steps_per_epoch=steps,
    epochs=epochs,
    validation_data = test_gen,
    validation_steps = test_steps,
    callbacks=callbacks
)

In [ ]:
model_json = face_model.to_json()
face_model.save_weights('model_weights.h5')
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
from tensorflow.keras.models import model_from_json
class FacialExpressionModel(object):
    EMOTIONS_LIST = ["Angry", "Disgust",
                    "Fear", "Happy",
                    "Neutral", "Sad",
                    "Surprise"]
    def __init__(self, model_json_file, model_weights_file):
        # load model from JSON file
        with open(model_json_file, "r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)
        # load weights into the new model
        self.loaded_model.load_weights(model_weights_file)
        self.loaded_model.make_predict_function()
    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]

In [ ]:
import cv2
facec = cv2.CascadeClassifier("drive/MyDrive/Colab Notebooks/Take 2/haarcascade_frontalface_default.xml/")
model = FacialExpressionModel("model.json", "model_weights.h5")
font = cv2.FONT_HERSHEY_SIMPLEX
class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(-1)
    def __del__(self):
        self.video.release()
    # returns camera frames along with bounding boxes and predictions
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)
        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]
            roi = cv2.resize(fc, (48, 48))
            pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])
            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)
        return fr

In [ ]:
def gen(camera):
    while True:
        frame = camera.get_frame()
        cv2.imshow('Facial Expression Recognization',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

In [2]:
gen(VideoCamera())